## Climate Change - WiDS Datathon 2022

The goal of this competition is to predict the energy consumption using building characteristics and climate and weather variables .

### Importing Libraries

In [1]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn import datasets
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.dummy import DummyRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, make_scorer

from sklearn.model_selection import (
    cross_val_score,
    cross_validate,
    train_test_split,
)

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.tree import DecisionTreeRegressor, export_graphviz
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import Lasso, LassoCV
from xgboost import XGBRegressor
from lightgbm.sklearn import LGBMRegressor
from catboost import CatBoostRegressor

%matplotlib inline

### Data Exploration

The WiDS Datathon 2022 focuses on a prediction task involving roughly 100k observations of building energy usage records collected over 7 years and a number of states within the United States. The dataset consists of building characteristics (e.g. floor area, facility type etc), weather data for the location of the building (e.g. annual average temperature, annual total precipitation etc) as well as the energy usage for the building and the given year, measured as Site Energy Usage Intensity (Site EUI). Each row in the data corresponds to the a single building observed in a given year. 

In [2]:
# Reading the Train data
train_df = pd.read_csv("data/train.csv", parse_dates=["year_built"])
train_df.head()

,Year_Factor,State_Factor,building_class,facility_type,floor_area,year_built,energy_star_rating,ELEVATION,january_min_temp,january_avg_temp,...,days_above_80F,days_above_90F,days_above_100F,days_above_110F,direction_max_wind_speed,direction_peak_wind_speed,max_wind_speed,days_with_fog,site_eui,id
0,1,State_1,Commercial,Grocery_store_or_food_market,61242.0,1942.0,11.0,2.4,36,50.5,...,14,0,0,0,1.0,1.0,1.0,NaN,248.682615,0
1,1,State_1,Commercial,Warehouse_Distribution_or_Shipping_center,274000.0,1955.0,45.0,1.8,36,50.5,...,14,0,0,0,1.0,NaN,1.0,12.0,26.500150,1
2,1,State_1,Commercial,Retail_Enclosed_mall,280025.0,1951.0,97.0,1.8,36,50.5,...,14,0,0,0,1.0,NaN,1.0,12.0,24.693619,2
3,1,State_1,Commercial,Education_Other_classroom,55325.0,1980.0,46.0,1.8,36,50.5,...,14,0,0,0,1.0,NaN,1.0,12.0,48.406926,3
4,1,State_1,Commercial,Warehouse_Nonrefrigerated,66000.0,1985.0,100.0,2.4,36,50.5,...,14,0,0,0,1.0,1.0,1.0,NaN,3.899395,4


In [3]:
# Reading the Test data

test_df = pd.read_csv("data/test.csv", parse_dates=["year_built"])

In [4]:
print("Number of train samples are",train_df.shape)
print("Number of train samples are",test_df.shape)

Number of train samples are (75757, 64)
Number of train samples are (9705, 63)


In [5]:
train_df.describe(include="all")

,Year_Factor,State_Factor,building_class,facility_type,floor_area,year_built,energy_star_rating,ELEVATION,january_min_temp,january_avg_temp,...,days_above_80F,days_above_90F,days_above_100F,days_above_110F,direction_max_wind_speed,direction_peak_wind_speed,max_wind_speed,days_with_fog,site_eui,id
count,75757.000000,75757,75757,75757,7.575700e+04,73920,49048.000000,75757.000000,75757.000000,75757.000000,...,75757.000000,75757.000000,75757.000000,75757.000000,34675.000000,33946.000000,34675.000000,29961.000000,75757.000000,75757.000000
unique,NaN,7,2,60,NaN,180,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,State_6,Residential,Multifamily_Uncategorized,NaN,1927.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,50840,43558,39455,NaN,2064,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,4.367755,NaN,NaN,NaN,1.659839e+05,NaN,61.048605,39.506323,11.432343,34.310468,...,82.709809,14.058701,0.279539,0.002442,66.552675,62.779974,4.190601,109.142051,82.584693,37878.000000
std,1.471441,NaN,NaN,NaN,2.468758e+05,NaN,28.663683,60.656596,9.381027,6.996108,...,25.282913,10.943996,2.252323,0.142140,131.147834,130.308106,6.458789,50.699751,58.255403,21869.306509
min,1.000000,NaN,NaN,NaN,9.430000e+02,NaN,0.000000,-6.400000,-19.000000,10.806452,...,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,12.000000,1.001169,0.000000
25%,3.000000,NaN,NaN,NaN,6.237900e+04,NaN,40.000000,11.900000,6.000000,29.827586,...,72.000000,6.000000,0.000000,0.000000,1.000000,1.000000,1.000000,88.000000,54.528601,18939.000000
50%,5.000000,NaN,NaN,NaN,9.136700e+04,NaN,67.000000,25.000000,11.000000,34.451613,...,84.000000,12.000000,0.000000,0.000000,1.000000,1.000000,1.000000,104.000000,75.293716,37878.000000
75%,6.000000,NaN,NaN,NaN,1.660000e+05,NaN,85.000000,42.700000,13.000000,37.322581,...,97.000000,17.000000,0.000000,0.000000,1.000000,1.000000,1.000000,131.000000,97.277534,56817.000000


In [6]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75757 entries, 0 to 75756
Data columns (total 64 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Year_Factor                75757 non-null  int64  
 1   State_Factor               75757 non-null  object 
 2   building_class             75757 non-null  object 
 3   facility_type              75757 non-null  object 
 4   floor_area                 75757 non-null  float64
 5   year_built                 73920 non-null  object 
 6   energy_star_rating         49048 non-null  float64
 7   ELEVATION                  75757 non-null  float64
 8   january_min_temp           75757 non-null  int64  
 9   january_avg_temp           75757 non-null  float64
 10  january_max_temp           75757 non-null  int64  
 11  february_min_temp          75757 non-null  int64  
 12  february_avg_temp          75757 non-null  float64
 13  february_max_temp          75757 non-null  int

### Data Preprocessing

We will be handling missing values and segregating our data

In [7]:
# Handling missing values - year_built, energy_star_rating, direction_max_wind_speed, direction_peak_wind_speed, max_wind_speed, days_with_fog



In [8]:
# Converting few columns to factor type

# train_df["year_built"] = train_df["year_built"].astype("object")
# train_df["Year_Factor"] = train_df["Year_Factor"].astype("object")
# test_df["year_built"] = test_df["year_built"].astype("object")
# test_df["Year_Factor"] = test_df["Year_Factor"].astype("object")

In [9]:
# Segregating different types of features

categorical_features = ["year_built", "State_Factor", "facility_type", "Year_Factor"]
binary_features = ["building_class"]
# ordinal_features = ["Year_Factor"]
drop_features = ["id"]
target = "site_eui"
numeric_features = list(
    set(train_df.columns) - 
    set(
        categorical_features +
        binary_features +
        drop_features +
        [target]
    )
)

In [14]:
# Building a Column Transformer

categorical_pipeline = make_pipeline(
    SimpleImputer(strategy = 'constant', fill_value = 'missing'),
    OneHotEncoder(handle_unknown="ignore"))

numeric_pipeline = make_pipeline(
    SimpleImputer(strategy = 'median'),
    StandardScaler())

preprocessor_all = make_column_transformer(
    (categorical_pipeline, categorical_features),
    (OneHotEncoder(handle_unknown="ignore", drop="if_binary"), binary_features),
    (numeric_pipeline, numeric_features),
    ("drop", drop_features)
)   

In [15]:
# from sklearn import preprocessing
# le = preprocessing.LabelEncoder()

# for i in train_df.columns:
#     if train_df[i].dtypes == 'object':
#         train_df[i] = le.fit_transform(train_df[i])
#         print(i)

In [16]:
X_train, y_train = train_df.drop(['site_eui'], axis = 1), train_df['site_eui']
# X_test, y_test = test_df.drop(['site_eui'], axis = 1), test_df['site_eui']

In [17]:
preprocessor_all.fit(X_train, y_train)

ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(fill_value='missing',
                                                                strategy='constant')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['year_built', 'State_Factor', 'facility_type',
                                  'Year_Factor']),
                                ('onehotencoder',
                                 OneHotEncoder(drop='if_binary',
                                               handle_unknown='ignore'),
                                 ['building_class']),
                                (...
                                  'march_max_temp', 'energy_star_rating',
                                  'may_max_temp', 'june_avg_temp',
 

In [18]:
# preprocessor_all.fit(X_train)

In [19]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75757 entries, 0 to 75756
Data columns (total 63 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Year_Factor                75757 non-null  int64  
 1   State_Factor               75757 non-null  object 
 2   building_class             75757 non-null  object 
 3   facility_type              75757 non-null  object 
 4   floor_area                 75757 non-null  float64
 5   year_built                 73920 non-null  object 
 6   energy_star_rating         49048 non-null  float64
 7   ELEVATION                  75757 non-null  float64
 8   january_min_temp           75757 non-null  int64  
 9   january_avg_temp           75757 non-null  float64
 10  january_max_temp           75757 non-null  int64  
 11  february_min_temp          75757 non-null  int64  
 12  february_avg_temp          75757 non-null  float64
 13  february_max_temp          75757 non-null  int

### Model Training

In [20]:
results = {}

In [21]:
# Adapting this function from previous labs to report mean cross validation scores of different models

def mean_std_cross_val_scores(model, X_train, y_train, **kwargs): 
    """ 
    Returns mean and std of cross validation 
 
    Parameters 
    ----------
    model : 
        scikit­learn model 
    X_train : numpy array or pandas DataFrame 
        X in the training data 
    y_train : 
        y in the training data 
 
    Returns 
    -------
        pandas Series with mean scores from cross_validation 
    """ 
 
    scores = cross_validate(model, X_train, y_train, **kwargs) 
 
    mean_scores = pd.DataFrame(scores).mean() 
    std_scores = pd.DataFrame(scores).std() 
    out_col = [] 
 
    for i in range(len(mean_scores)): 
        out_col.append((f"%0.5f (+/­ %0.5f)" % (mean_scores[i], std_scores[i]))) 
 
    return pd.Series(data=out_col, index=mean_scores.index)

In [22]:
def root_mean_squared_error(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

In [23]:
scoring_metrics = make_scorer(root_mean_squared_error)

#### Baseline Model (Dummy Regressor)

In [24]:
dummy_regressor_pipe = DummyRegressor()
results['Dummy Regressor'] = mean_std_cross_val_scores(dummy_regressor_pipe, X_train, y_train, scoring = scoring_metrics, return_train_score = True)
pd.DataFrame(results)

,Dummy Regressor
fit_time,0.00942 (+/­ 0.00642)
score_time,0.00026 (+/­ 0.00012)
test_score,58.09218 (+/­ 7.17895)
train_score,58.21226 (+/­ 1.76878)


#### Linear Model (Ridge)

In [25]:
# ridge_pipe = make_pipeline(preprocessor_all, Ridge())
# results['Ridge'] = mean_std_cross_val_scores(ridge_pipe, X_train, y_train, scoring = scoring_metrics, return_train_score = True)
# pd.DataFrame(results)

In [26]:
pipe_random_forest = make_pipeline(preprocessor_all, RandomForestRegressor(random_state = 123))
pipe_xgboost = make_pipeline(preprocessor_all, XGBRegressor(random_state = 123,  eval_metric="logloss", verbosity=0))
pipe_lgbm = make_pipeline(preprocessor_all, LGBMRegressor(random_state = 123))
pipe_catboost = make_pipeline(preprocessor_all, CatBoostRegressor(random_state = 123, verbose=0))
models = {
#     'Random Forest' : pipe_random_forest,
    'XGBoost' : pipe_xgboost,
    'LGBM' : pipe_lgbm,
    'CatBoost' : pipe_catboost
}

In [27]:
for model in models:
    results[model] = mean_std_cross_val_scores(
        models[model],
        X_train,
        y_train,
        cv=5,
        n_jobs=-1,
        scoring=scoring_metrics,
        return_train_score=True
    )
    
pd.DataFrame(results)

,Dummy Regressor,XGBoost,LGBM,CatBoost
fit_time,0.00942 (+/­ 0.00642),23.39668 (+/­ 0.33036),2.88415 (+/­ 1.11496),23.17379 (+/­ 0.87731)
score_time,0.00026 (+/­ 0.00012),0.46989 (+/­ 0.07419),0.21230 (+/­ 0.06144),0.16020 (+/­ 0.03477)
test_score,58.09218 (+/­ 7.17895),57.08715 (+/­ 27.14458),52.77631 (+/­ 15.59953),52.01764 (+/­ 16.20664)
train_score,58.21226 (+/­ 1.76878),33.03260 (+/­ 1.22405),40.07848 (+/­ 1.36754),34.88659 (+/­ 1.30750)


In [28]:
# test_x = test_df.drop(['id'], axis=1)
# test_x.shape

In [39]:
pipe_lgbm.fit(X_train, y_train)
predict_sub = pipe_lgbm.predict(test_df)
predict_sub

array([267.29865755, 208.58424571, 254.84884013, ...,  31.71735822,
        41.15880974,  40.52370082])

#### Submission for Kaggle

In [40]:
sub = pd.read_csv("data/sample_solution.csv")
sub

,id,site_eui
0,75757,0.0
1,75758,0.0
2,75759,0.0
3,75760,0.0
4,75761,0.0
...,...,...
9700,85457,0.0
9701,85458,0.0
9702,85459,0.0
9703,85460,0.0


In [41]:
sub.site_eui = predict_sub

In [42]:
sub.to_csv("submission_SJ.csv", index=False)

In [43]:
print("Submission downloaded")

Submission downloaded
